In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_pickle(
    "../../data/paired_spectra/casmi2022/processed_massformer/spec_df.pkl"
)
df_h = df[df["prec_type"] == "[M+H]+"]
print(len(df_h))
df_h.head()
smiles_h = df_h["smiles"].to_list()

In [ ]:
df_h["name"] = "casmi2022_" + df_h["casmi_id"].astype(str)
df_h.reset_index(drop=True, inplace=True)
df_h.head()

In [ ]:
for smi in smiles_h:
    print(smi)

In [ ]:
cls_file = "../../data/paired_spectra/casmi2022/classyfire_casmi.csv"
df_cls = pd.read_csv(cls_file)
df_cls.head(40)

In [ ]:
# remove rows where ChemOntID is NotAvailable
df_cls = df_cls[df_cls["ChemOntID"] != "NotAvailable"]
df_cls.head()

In [ ]:
# split classifiedResults column on :
df_cls[["level", "value"]] = df_cls["ClassifiedResults"].str.split(
    ": ", expand=True
)
# remove rows where level is alternative parent
df_cls = df_cls[df_cls["level"] != "alternative_parents"]
df_cls.head(20)

In [ ]:
# compress df_cls on CompoundID by creating column for each level
levels = df_cls["level"].unique()
ids = [int(id.split("-")[-1]) - 1 for id in df_cls["CompoundID"].unique()]
df_comp = pd.DataFrame(index=ids, columns=levels)

for i, row in df_cls.iterrows():
    df_comp.loc[int(row["CompoundID"].split("-")[-1]) - 1, row["level"]] = row[
        "value"
    ]

df_comp.head(20)

In [ ]:
# insert rows to make id increment by 1
missing_ids = set(np.arange(1, len(df_h), 1)) - set(df_comp.index.astype(int))
missing_df = pd.DataFrame(index=list(missing_ids), columns=levels)
df_complete = pd.concat([df_comp, missing_df])
df_complete.index = df_complete.index.astype(int)
df_complete.sort_index(inplace=True)

print(len(df_complete))
df_complete.head(30)

In [ ]:
# join df_h and df_complete on index
casmi_cls = df_h.join(df_complete, how="left")
print(len(casmi_cls))
casmi_cls.head()

In [ ]:
casmi_cls[levels] = casmi_cls[levels].fillna("Unclassified")

In [ ]:
casmi_cls.to_pickle(
    "../../data/paired_spectra/casmi2022/casmi_classyfire_results.pkl"
)

In [ ]:
casmi_cls["Superclass"].value_counts()

In [ ]:
casmi_cls["Class"].value_counts()